<h1 style="color:black" align="center">Линейная классификация</h1>

Мы помним что функционал ошибки выглядел следующим образом:

$\large Q(a, X) = \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [a(x_i) = y_i]$ - долей правильных ответов (accuracy)

Нам будет удобнее решать задачу минимизации, поэтому будем вместо этого использовать долю неправильных ответов:

$\large Q(a, X) = \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [a(x_i) \ne y_i]$ - доля ошибок.

Так как этот функционал не дифференцируемый, то его необходимо заменить на функцию, которая оценивает сверху идикатор ошибки. Записали оценку сверху мы следующим образом:

$\large 0 \le \frac{1}{\ell} \sum\limits_{i=1}^{\ell} [y_i \langle w, x_i \rangle < 0] 
\le 
 \frac{1}{\ell} \sum\limits_{i=1}^{\ell} \tilde L (y_i \langle w, x_i \rangle) \to \underset{w}{\text{min}}$

<h1 style="color:#008B8B">1. Логистическая регрессия</h1>

<h2 style="color:#008B8B">1.1 Оценивание вероятностей</h2>

Необходимо, чтобы модель оценивала веротяность того, что объект положителен.

## Для чего необходимо оценивать вероятность?

Предположим, имеется модель $\text{b(x)} \in [0, 1]$. Представим, что модель выдала на некотором объекте веротяность равную $0.8$ в классе +1. 

**Как это можно интерпретировать?**

Например, имеется множество из 10 объектов, где модель возвращает приблизительно одну веренность в своём прогнозе, тогда мы ожидаем, что среди этих объектов 80% будут положительными.

Предположим, мы построили модель, кторая оценивает вероятность положительного класса. Записывать это будем следующим образом:

$\large b(x) \approx p(y = +1 | x)$ - модель оценивает то, что класс положительный для данного объекта $x$. Оценивание веротяности принадлежности объекта к некоторому классу позволяет посмотреть уверенность модели в своём прогнозе. Также бывают задачи, когда нет понимания того, какой ответ в задачи: 

### Пример 1

Пример, мы решаем задачу предсказания кликов пользователя по рекламе, где необходимо предсказать кликнет ли пользователь по данному рекламному банеру. Имеются следующие 3 категориальных признака по которым мы предсказываем нажмёт ли пользователь на рекламный банер.

Так как объект описывается тремя идентификаторами, то может быть ситуация, когда пользователь отправлят один и тот же запрос на который показывается один и тот же банер:

|id пользователя | id запроса | id банера | Нажал ли пользователь? |
|:--------------:|:----------:|:---------:|:----------------------:|
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |0                       |
|10              |50          |527        |1                       |

Пользователь может в большинстве случаев не нажимать на рекламу, спустся определённое количество запросов он кликнул на данный банер. Проблема заключается в том что один объект встречается много раз с рзными целевыми переменным (большенство раз 0). Если данную обучающую выборку передать классификатору, который возвращает только класс объекта, тогда ему будет тяжело извлечь закономерность из этой выборки, так как объект один, а ответы целевой переменной разыне. Тогда, можно сделать так, чтобы классификатор возвращал не класс, а вероятность пренадлежность объекта к некоторому классу.

### Пример 2

Представим, что пользователь вводит некоторый запрос, на который необходим показать один из 4 банера. За каждый кликнутый банер мы получаем определённую сумму денег, обазначим как $c(x)$. Показывать самый дорогой банер будет плохой идей, так как пользователь может на него не нажать, для этого необходимо ввести вероятность клика $p(y = +1| x)$ которую оценивает модель.

Что мы максимизируем в данной задаче? Необходимо максимизировать математическое ожидание прибыли, а что такое мат.ожидание прибыли? Это $с(x) \cdot p(y = +1| x)$

|$$c(x)$$|$$p(y = +1| x)$$ |$$с(x) \cdot p(y = +1| x)$$ |
|:------:|:---------------:|:--------------------------:|
|1       |0.5              |0.5                         |
|5       |0.1              |0.5                         |
|10      |0.3              |3                           |
|100     |0.01             |1                           |

Лучшее математическое ожидание будет $3$. Значит данный банер лучше всего показывать. Соответсвенно, данная вероятность помогает выбрать то, что необходимо показывтаь пользователю.

### Пример 3

Банк принимает решение того, какой продукт необходимо предложить пользователю. Имеется продукт мобильный банк, который он подключит с вероятностью 0.9, либо мы можем предложить пользоватлею кредит, на который он согласится с вероятностью 0.1. Если предлогать пользователю продук по вероятности, тогда мы не сможем узнать цену рекомендации. Если на основе вероятности оценить то, сколько мы заработаем, тогда предложить крдеит пользвателю будет более хорошей идеей, так как прибыль намного будет больше.

## Что такое оценивание вероятностей?

Предположим, имется набор равных объектов $\{x_1, \ldots, x_n\}$ (мы считаем что они равны, в реальности они должны быть в некоторой окрестности), который устроент так, что модель вовращает одинаковы числа на этих объетках $b(x_1) = b(x_2) = \ldots = b(x_n)$ (так как объекты равны), а целивая переменная на данных объектах может имеет разные значения $\{y_1, \ldots, y_n\}$.

Так как мы хотим чтобы модель возвращала вероятности того, что объект является положительным, поэтому мы хотим, чтобы модель выдала долю положительных среди этих объектах и это будет хорошей оценкой вероятности. Это требование можно записать через функцию потерь:

$\large b(x_1) = \frac{1}{n} \sum\limits_{i=1}^n {[y_i = +1]}$

$\large \underset{b \in \mathbb{R}}{\text{argmin}} \frac{1}{n} \sum\limits_{i=1}^n L(y_i, b)$ - ищем такое $b$ из всех возможных, которое даёт минимальную среднюю ошибку на $n$ объектах. Где $b$ - это прогноз модели на $n$ объектах в конкретной точке $x$, тем самым $b$ - это число.

Мы хотим, чтобы при минимизации функции потреь усредненной по всем объектам ее значение совпадало с долей положительных объектов:

$\large \underset{b \in \mathbb{R}}{\text{argmin}} \frac{1}{n} \sum\limits_{i=1}^n L(y_i, b) \approx \frac{1}{n} \sum\limits_{i=1}^n [y_i = +1]$

Это условие накладывается на функцию потерь, которое требует чтобы модель на $n$ объектах выдала один и тот же ответ, тогда оптимально ответ будет равен доли положительных среди этих объектов.

### Устремление $n \to \infty$

Стоит подметить, что модель выдаёт один и тот же прогноз на множестве $\{x_1, \ldots, x_n\}$ объектов, мощность которого $n$ (это не все возможные объекты).

Запишем более точную формулировку, так как мы работаем с вероятностью. Для этого будем считать, что $n \to \infty$, другими словами, имеется некоторая область (множество) из всех возможных объектов, где бесконечное количество объектов и модель на них возвращает одинаковый прогноз. Тогда посчитаем математическое ожидание функции потерь на объекте $x$ и мы требуем, чтобы $\text{argmin}$ данного математического ожидания достигался на вероятности положительного класса для данного объетка $p(y = +1 | x)$

$\large \underset{b \in \mathbb{R}}{\text{argmin}} ~ \mathbb{E} (L(y, b)|x) = p(y = +1 | x)$ - условие на функцию потерь корректного оценивания вероятностей.

Если функция потерь удовлетворяет данному требованию, тогда оптимальными выходами модели для данной функции потерь будут оценки вероятности. Но так как модели ограничены по сложности, например линейная модель не может в двух близких точках пространства выдать разные прогнозы, поэтому, если мы обучаем некую модель на данную функцию потерь, тогда модель будет пытаться оценивать вероятности, но не факт что у нее это везде получится и чем более сложную модель мы выберем, тем больше шансов что модель будет выдавать что-то похожее на оценки вероятности. Тем самы, если мы обучаем логистическую регрессию, означает лишь то, если сможет, тогда выдаст корректные оценки вероятности, но не факт, что она выдаст корректные оценки вероятности.

Если данное условие выполнено, тогда функция потерь требует от модели, чтобы она оценивала вероятности. Давайте посмотрим на эту задачу с другой стороны и возьмем подмножество объектов $C \in X$ (объекты уже не совпадают, так как в множестве те объекты, где модель выдает примерно один ответ), такое что внутри этого подмножества модель выадает примерно одинаковые значения $b(x) \approx p$. И тогда, у нас есть некоторое количество объектов, где модель возвращает одно и то же $b(x_1) \approx  b(x_2) \approx  \ldots \approx  b(x_n)$.

Запишем функционал ошибки:

$\large Q(b, X) = \text{Объекты не из области C} ~ + \sum\limits_{x_i \in C}^n{L(y_i, p)} \to \underset{}{\text{min}}$  - В функции потерь вместо прогноза $x_i$ стоит $p$, так как ответ модели примерно равен $p$.

Тем самым в функционале ошибки будет подмножество, где модель выдает одно и то же значение и фактически соответсвует записанному случаю, и если функци потерь удовлетворяет записанному требованию, тогда мы будем требовать чтобы модель на данном подмножестве объектов выдала корректную оценку вероятности.

**Переход слева и справа**

Правая часть неравенства переходит вероятность того, что мы получаем положительный объект в точке $x$, другими словами мы берем бесконечное количество объектов в точке $x$ (с одинаковыми значениями признаков) и смотрим какая доля из них положительная - это и есть оценка вероятности.

Левая часть переходит в математическое ожидание функции потерь $y$ в данной точке $x$, которое мы уже расписписали по формуле ниже и это то же самое что сумма в формуле без перехода к бесконечности.

**Переход от первой формулы к второй**

В первой формуле предпологается, что есть конечное множество объектов, которых $n$ штук. Функция потерь устроена так, что если мы будем искать оптимальную константу $b$ для прогноза на $n$ объектах, тогда оптимальное значение $b$ должно быть примерно равно доле положительных объектов в этом множестве. Значит, мы требуем для множества объектво где модель выдаёт один и тот же прогноз, чтобы этот прогноз был примерно равен вероятности положительного класса.

Переход к второй формуле. Мы говорим, что $n$ теперь стремится к бесконечности. Имеется один объект $x$, экземпляров которого большое количество, а именно $n$ раз данный объект встречается в выборке и на этом объекте разные значения целевой переменной. Тогда, мы считаем математическое ожидание функции потреть по распределению $y$ в данной точке и минимальное математическое ожидание должно достигаться при константе $b$ равной вероятности положительного класса.

$b$ оценивает вероятность положительного класса, поэтому $b$ должно аппроксимировать веротяность положительного класса. 

**Про математическое ожидание**

Математическое ожидание берётся по распределению $y$ при условии данного $x$ $p(y|x), где $$y \in \{-1, +1\}$. Математическое ожидание будет иметь следующий смысл:

Это вероятность того, что $y$ равен +1 при условии $x$...

$\large \mathbb{E} (L(y, b)|x) = p(y=+1 |x) \cdot L(+1, b) + p(y = -1 | x) \cdot L(-1, b)$ - математическое ожидание это сумма по всем возможным исходам $y$


Почему записали математическое ожидание? Так как мы $n$ устремляем в бесконечность, тогда вместо средней ошибки можно записать математическое ожидание ошибки.  А дальше, по закнону больших чисел в первой формуле, доля положительных объектов будет стремится к вероятности положительного класса. Мы дальше во второй формуле требуем строгое равенство, чтобы функция потерь была устроена так, что для точки с бесконечным количеством объектов оптимально выдавать прогноз равный вероятности положитлеьного класса.

**Дополнительные вопросы**

Что означает запись при условии $x$? Это означает, что мы рассматриваем распределение ответов конкретно для данного объекта. Допусим, имется объект $x_1$ на котором вероятность положитлеьного класса $p(y = +1 |x) = 0.9$, есть другой объект $x_2$, где вероятность равна $p(y = +1 |x) = 0.4$. Получается, что это вероятность того, что данный объект $x$ положительный.

Что означает распределение ответов для одного объекта? Мы значем что объект один, но этот объект может встречаться несколько раз, это мы рассаматривали выше. Предположи, есть пространство объектов, где много положителных и среди них в середине расположен один отрицательный, тогда вероятнотность того, что этот объект будет отрицательным в данной области (множестве) будет небольшой. Поскольку эти объекты расположены рядом и модель вовзращает практически идентичный прогноз, тогда мы ожидаем, что вероятность того, что этот объект отрицательный будет небольшой. 

Под объекта расположенными рядом понимается то, что объекты с практически одинаковым признаковым описанием, которые группируются в признаковом простарнстве (мы это называли множеством), тогда логично,что скорее всего они относятся к одной группе. Соответсвенно записи формул по сумме $n$ объектов понимается что, имеется некая группа мощностью $n$, где их признаковое описание практически похоже, значит и модель будет выдавать приблизительно один резултат.

### Пример 4 Функционал для квадрата ошибки

Применим требование для квадратичной ошибки.

$\large L(y, b) = (y-b)^2$

Подставив в левую часть условия на функцию потерь:

$\large \underset{b \in \mathbb{R}}{\text{argmin}} ~ \mathbb{E} (L(y, b)|x) = p(y = +1 | x)$

Посмотрим, что будет оптимальное для такой ошибки, если у нас имеется много объектов с одинаковым признаковым описанием:

$\large \mathbb{E} ((y - b)^2 | x) = p(y = +1 |x) \cdot (1 - b)^2 + p(y = 0|x) \cdot (-b)^2$

Для того, чтобы требование выполнялось для $MSE$, необходимо пронумировать классы $y \in \{0, 1\}$. 

$p(y = +1 |x) \cdot (1 - b)^2$ - вероятность того, что класс положительный;

$p(y = 0|x) \cdot (-b)^2$ - вероятность того, что класс отрицательный.

Найдём минимум математического ожидания (argmin):

$\large \frac{\partial}{\partial b} = 2 \cdot p(y = +1 |x) (b-1) + 2p(y=0|x)b = $

Заметим, что $p(y=0|x) = (1-p(y = +1|x))$. Для краткости заменим $p(y = +1|x) = p$.

$\large = 2pb -2p + 2b - 2pb = 0$

Получаем, что $\large b = p(y = +1|x)$ - оптимально для модели (константы) быть равной вероятности положительного класса. 

Тем самым, для квадратичной фукнции потерь выполняется требование, что argmin этого математического ожидания равен вероятности положительного класса. Значит, мы можем использовать квадратичную функцию потерь для моддели, которая будет корректно оценивать вероятности.

**Как это можно интерпретировать?** Что такое вероятность положительного класса для данного объетка? Это доля положительных экземпляров среди объектов, если экземпляров данного объетка много. После эту долю мы умнжаем на ошибку в случае, если прогноз $b$ на данный экземплярах. Дальше долю отрицательных положительных экземпляров умножаем на ошибку при прогнозе $b$ для отрицатлеьных объектов и смотрим оптимальное значение $b$, оказыватся что опитмально брать $b$ в качестве вероятности положительного класса. Получается, если мы рассматриваем среднеквадратичную ошибку, тогда она устроена так, что для ней оптимально выдавать в некоторой области долю положитлеьных объектов.

**Проблема обучения**

$\large \frac{1}{\ell} \sum\limits_{i=1}^{\ell} {(\langle w, x_i \rangle - y_i)^2} \to \underset{w}{\text{min}}$, где $y \in \{0, 1\}$

Мы знаем, что большой отступ для модели это хорошо, когда она ставит объекты далеко от разделяющей гиперплоскости. Проблема заключается в том, что функционал модели будет запрещать ставить объекты далеко от разделяющей гиперплоскости. Если $y_i = 1$, а скалярное произведение на этом объекте $\langle w, x_i \rangle = 10^6$, тем самым объект находится с правильной стороны гиперплоскости при этом очень далеко, но за это будет большой штраф для модели. 

### Функционал для квадрата ошибки

Мы проверили, что квадарт ошибки $(y - b)^2$ обладает нужным свойством, поэтому его можно использовать для обучения классификатора который оцениват уверенность. Толко должно принадлежать $b \in \{0, 1\}$, если взять $b(x) = \langle w, x \rangle$ равное скалярному произведению, тогда не сходистя, ведь скалярное произведение возвращает любое вещественное число, для решения это проблемы можно придумать преобразование скалярного приозведения чтобы оно переводило вещественно число в отрезок от 0 до 1. Мы будем использовать для этого сигмойду:

$\large \sigma (z) = \frac{1}{1 + e^{-z}}$

Тогда обучать линейный классификатор будем на следующем функционале:

$\large \frac{1}{\ell} \sum\limits_{i=1}^{\ell} (\sigma (\langle w, x_i \rangle) - y_i)^2 \to \underset{w}{\text{min}}$

Вот мы записали модель, которая будет оценивать верояность.

**Чем плохой этот функционал?**

Сигмойда быстро достигает 0 или 1 и становится практически константой. Предположим что на объекте  $x_1$, где целевая переменная равана $y=1$ веса имеют такое значение, что их скалярное произведение даст $\langle w, x_1 \rangle = -1000$, тогда $\sigma(\langle w, x_1 \rangle) \approx 0$. Ну и видим, что ошибка получается огромной.

Допустим мы используем стохастический градиентый спуск и считаем градинт функционала по $w$:

$\large \nabla_w (\sigma(\langle w, x_1 \rangle) - y_1)^2= (\sigma(\langle w, x_1 \rangle) - y_1)^2 \cdot \sigma'(\langle w, x_1 \rangle) \cdot x_1$

Здесь мы видим производную сигмойды $\sigma'(\langle w, x_1 \rangle)$ в точке примеро равной $-1000$, а чему будет равна производна в этой точке если посмотреть на график? Конечно же примерно нулую, тогда и градиент тоже будет примерно равен нулю.

Получается, если мы ошибаемся на объекте и попадаем константную зону сигмойды, тогда градиентый спуск не сможет оттуда выйти, это называется затуханием градиента. Поэтому использовать $MSE$ для обучения такой модели будет плохой идеей.

### Пример 5

Если првоерить данное свойство для абсолютной ошибки:

$\large L(y, b) = |y - b|$

Опитмальное $b\in \{0, 1\}$

### Что мы рассмотрели выше?
Пусть выполнено следующее свойство:

$\large \underset{b \in \mathbb{R}}{\text{argmin}} =\mathbb{E} (L(y, b)|x) = p(y = +1 | x)$

1. Если один объект $x$ встречается много раз в выборке и нет других объектов  в выборке, тогда модель $b(x)$ будет оценивать вероятность положительного класса. (такуой случай не встречается на практике, он теоретический)

2. Если много разных объектов в выборке и нет ограничений на прогноз в каждом из них (модель может быть любой функцией, где на каждом объекте она может вернуть любое чилсо), тогда нет гарантий что данное свойство будет выполняться. 

3. Если много разных объектов в выборке и прогозны по объектам ограничиваются видном модели. Это означает, например, если некоторая метрика расстояния между двумя объектами небольшая, тогда выходы объектов примерно равны на этих двух объектах:

    $\large p(x_1, x_2) < \epsilon \Longrightarrow b(x_1) \approx b(x_2)$
    
    Из-за того, что выходы модели связаны на объекта, тогда свойство будет выдавать приблизительно тоже самое. Хотя точно похожих объектов нет, но есть объекты, на которых выходы можеди похожи, тогда изходя из свойства, мы будем примено оценивать их вероятность.

<h2 style="color:#008B8B">1.2 Правдоподобие и логистические потери</h2>

Имеются объекты $x_1, \ldots, x_{\ell}$ с правильными ответами $y_1, \ldots, y_{\ell}$, где $y \in \{-1, +1\}$, а $b(x) \in [0, 1]$. А модель оценивает веротяность того, что объект относится к положительнмоу классу $b(x_1), \ldots, b(x_{\ell})$. Для каждой точки известна веротяность того, что этот объект положительный (с точки зрения модели). Тогда запишем правдоподобие, которое говорит о том, насколько модель согласуется с данными:

$\large Q(a, X) = \prod\limits_{i = 1}^{\ell} b(x_i)^{[y_i = +1]} (1 - b(x_i))^{[y_i = -1]} \to \underset{b}{\text{max}}$

Мы проходимся по всем объектам, если объект положителен, тогда максимизируем $b(x_i)^{[y_i = +1]}$, то есть вероятность того, что это положительный класс должна быть близка к еденице, так как целевая переменная имеет значение 1. Если объект отрицателен, тогда вероятность положительного класса на этом обоъекте должна быть близка к нулю $b(x_i)^{[y_i = -1]}$.

Правдоподобие измеряет насколько соответсвуют выходы модели правильным ответам. Правдоподобие будет максимальным, равным одному, если для каждого положитлеьного объекта $b(x) = 1$, а для каждого отрицательного $b(x) = 0$.

Если в выборке все объекты положительные, тогда мы перемножим первый множетель и будем максимизировать, значит мы будем требовать, чтобы модель на всех объектах значение близкое к 1. А если все объекты в выборке отрицательные, тогда модель должна на всех объектах выдавать значение бликое к 0. Тем самым эта фукнция требует чтобы для положительных объектов вероятности былка как можно больше $b(x_i)$, а для отрицательных объектов $(1 - b(x_i))$ как можно меньше.

**Недостаток**

Проблема данного произведения заключается в том, что оно будет очень маленьким, что нам не хватит знаков после запятой. Если перемножить миллион объектов, где $b(x)$ принимают значения 0.7, 0.5 и тому подобные, тогда будет очень маленькое число. И поэтому мы логарифмируем, чтобы могли найти итоговое значение.

### Логарифмирование Log-Loss

Данное правдоподобие можно использовать как функционал для обучения алгоритма, с той лишь оговоркой, что удобнее оптимизировать его логарифм, который позволяет убрать произведение чисел от 0 до 1, а миниус нужен для замены максимизации на минимизацию:

$\large -\sum\limits_{i = 1}^{\ell} \biggl([y_i = +1] \log b(x_i) + [y_i = -1] \log (1 - b(x_i)) \biggl) \to \underset{b}{\text{min}}$

Функция потреь записывается следующим образом:

$\large L(y, b) = -[y_i = +1] \log b - [y = -1] \log (1-b)$ - log-loss

Данная функция потерь называется логарифмической (log-loss). Покажем, что она также позволяет корректно предсказывать вероятности. Запишем матожидание функции потерь в точке $x$:

$\large \mathbb{E} \biggl[L(y, b) | x \biggr] = \mathbb{E} \biggl[-[y = +1] \log b - [y = -1] \log(1 - b) | x \biggr] =$
$\large -p(y = +1 | x) \log b - (1 - p(y = +1 | x)) \log(1 - b)$

Продифференцируем по $b$:

$\large \frac{\partial}{\partial b} \mathbb{E} \biggl[L(y, b) | x \biggr] =
    -\frac{p(y = +1 | x)}{b} + \frac{1 - p(y = +1 | x)}{1 - b} = 0 \Longrightarrow  b = p(y = +1|x)$

Получаем, что $\large b = p(y = +1|x)$ - оптимально для модели (константы) быть равной вероятности положительного класса. И эту функцию потреть можно уже использовать для оптимизации модели.

<h2 style="color:#008B8B">1.3 Логистическая регрессия</h2>

Тогда $\large b(x) = \sigma(\langle w, x \rangle)$ и подставим это $b(x)$ в log-loss:

$\large -\sum\limits_{i = 1}^{\ell} \left([y_i = +1] \log \frac{1}{1 + \exp(-\langle w, x_i \rangle)} + [y_i = -1] \log \frac{\exp(-\langle w, x_i \rangle)}{1 + \exp(-\langle w, x_i \rangle)} \right) =$

Разделим числитель и знаменатель на числитель, после перевернём дробь:

$\large \log \frac{\exp(-\langle w, x_i \rangle)}{1 + \exp(-\langle w, x_i \rangle)} = \log \frac{1}{1 + \exp(\langle w, x_i \rangle)}= \log 1 + \exp(\langle w, x_i \rangle)$ 

Логарифм дроби это минус логарифм перевёрнутой дроби, тогда внесём минус внутрь:

$\large = \sum\limits_{i = 1}^{\ell} \biggl([y_i = +1] \log (1 + \exp(-\langle w, x_i \rangle)) + [y_i = -1] \log (1 + \exp(\langle w, x_i \rangle) \biggl)=$

Можем заметить, что эти два слогаемых одинаковые, только под экспонентой одного стоит минус, а другого плюс. Мы можем убрать идикатор и записать всё это как единое выражение внеся вутрь экспоненты $-y_i$. Тогда если $y_i$ положителен, то получаем логариф от экспоненты... минус скалряное произведение, как в первом слагаемом, а если $y_i$ отрицательный, тогда то получаем логариф от экспоненты... плюс скалярное произведение как во втором слогаемом. Значит, это две идентичные записи:

$\large = \sum\limits_{i = 1}^{\ell} \log \left(1 + \exp(-y_i \langle w, x_i \rangle) \right)$

И эту функцию потерь мы записывали, которую мы называли логистической. Здесь есть отступ. У этой функции все нормально с градиентом. Этот метод называется логистической регрессией. И свойство логистической регрессии заключается в том, что модель старается корректно оценивает вероятности.

**Почему используем сигмойду?**

Мы можем использовать другие функции. Почему сигмойда имеет здесь смысл? 

Так как мы пытаемся оценить того, что объект является положительным $\sigma(\langle w, x \rangle) \approx p(y = +1 | x)$. Если из данной формулы выразить скалярное произведение подставив сигмойду, тогда мы получим следующую формулу:

$\large \langle w, x \rangle = \log \frac{p(y = +1 | x)}{p(y = -1 | x)}$ - называется log-odds (логарифмические шансы).

Скалярное произведение оценивает логарифм, вероятности того, что объект положителен поделенное на вероятность того, что объект отрицателен. Если вероятности совпадают, то есть объект с равной вероятностью относится к положительному классу и отрицательному к, тогда мы получим еденицу (1/2 разделить на 1/2 равно 1) и есил взять логарифм, тогда мы получим ноль. Чем распределение более вырожденое, если вероятность того, что объект положителен приближается к 1, тогда данная дробь будет устремляться к бесконечности. Если вероятность того, что объект отрицателн повышается, тогда дробь будет стремится к 0, а log будет стремиться к минус бесконечности.

И получается, что скалярным произведением мы оцениваем шансы того, что объект является положительным.

### Пробит-регрессия

$b(x) = \Phi(\langle w, x \rangle)$, где $\Phi$ - это фкнция распределения для стандартного нормального распределения $\mathcal{N}(0, 1)$. И функция распределения тоже удовлетворяет тому свойству, что переводит вещественную прямую на интервал от 0 до 1, тем самым $\Phi$ будет давать корректное преобразование. Можно првоерить, что это будет тоже давать корректные оценки вероятности.

**Проблема с обучением**

Функция распределения для нормального распределения считается через интералл. Если это $\Phi$ подставить в log-los, тогда градиент будет считаться как градиент некотого интегралла и это будет сложно в вычислениях. 

**Откуда название?**

И это будет называться пробит-регрессией. Почему это так называется? Поскольку мы передаем скалярное произведение в функцию распределения, тогда можно считать что аргументы фукнции распределения измеряются в probability units, то есть какие-то еденицы, которые мы передаем в функцию распределения и поулчаем то, что она выдает. Посколькую $\langle w, x \rangle$ это  probability units, из названия получается пробит-регрессия.

Если мы берем сигмойду, тогда в сигмойду передается $\langle w, x \rangle$ тогда оно измеряется в log units - поэтому и логистическая регрессия.

<h1 style="color:#008B8B">2. Метод опорных векторов</h1>

Рассмотрим теперь другой подход к построению функции потерь,
основанный на максимизации зазора между классами.
Будем рассматривать линейные классификаторы вида

$\large a(x) = sign (\langle w, x \rangle + b), \qquad w \in \mathbb{R}^d, b \in \mathbb{R}$

<h2 style="color:#008B8B">2.1 Линейно разделимый случай</h2>

Будем считать, что существуют такие параметры $w_*$ и $b_*$,
что соответствующий им классификатор $a(x)$ не допускает ни одной ошибки
на обучающей выборке.
В этом случае говорят, что выборка линейно разделима.

Пусть задан некоторый классификатор $a(x) = sign (\langle w, x \rangle + b)$.
Заметим, что если одновременно умножить параметры $w$ и $b$ на одну и ту же положительную константу, то классификатор не изменится. Распорядимся этой свободой выбора и отнормируем параметры так, что

$\large \underset{x \in X}{\text{min}} | \langle w, x \rangle + b| = 1$

Можно показать, что расстояние от произвольной точки $x_0 \in \mathbb{R}^d$ до гиперплоскости, определяемой данным классификатором $a$, равно

$\large \rho(x_0, a) = \frac{|\langle w, x \rangle + b|}{\|w\|_2}$ - расстояние от точки до гиперплоскости

Тогда расстояние от гиперплоскости до ближайшего объекта обучающей выборки равно

$\large \underset{x \in X}{\text{min}} \frac{|\langle w, x \rangle + b| }{\|w\|_2 } = \frac{1}{\|w\|_2} \underset{x \in X}{\text{min}} |\langle w, x \rangle + b| = \frac{1}{\|w\|_2}$

Данная величина также называется отступом (margin).

Таким образом, если классификатор без ошибок разделяет обучающую выборку, то ширина его разделяющей полосы равна $\frac{2}{\|w\|_2}$. Известно, что максимизация ширины разделяющей полосы приводит к повышению обобщающей способности классификатора. Вспомним также, что на повышение обобщающей способности направлена и регуляризация, которая штрафует большую норму весов - а чем больше норма весов, тем меньше ширина разделяющей полосы.

Итак, требуется построить классификатор, идеально разделяющий обучающую выборку, и при этом имеющий максимальный отступ. Запишем соответствующую оптимизационную задачу, которая и будет определять метод опорных векторов для линейно разделимой выборки (hard margin support vector machine):

$\large \begin{equation}
\label{eq:svmSep}
    \left\{
        \begin{aligned}
            & \frac{1}{2} \|w\|^2 \to \min_{w, b} \\
            & y_i \left(
                \langle w, x_i \rangle + b
            \right) \geq 1, \quad i = 1, \dots, \ell.
        \end{aligned}
    \right.
\end{equation}$

Здесь мы воспользовались тем, что линейный классификатор дает правильный ответ
на объекте $x_i$ тогда и только тогда, когда $y_i (\langle w, x_i \rangle + b) \geq 0$. Более того, из условия нормировки $(2.1)$ следует, что $y_i (\langle w, x_i \rangle + b) \geq 1$.

В данной задаче функционал является строго выпуклым, а ограничения линейными,
поэтому сама задача является выпуклой и имеет единственное решение. Более того, задача является квадратичной и может быть решена крайне эффективно.

<h2 style="color:#008B8B">2.2 Неразделимый случай</h2>

Рассмотрим теперь общий случай, когда выборку невозможно идеально разделить гиперплоскостью. Это означает, что какие бы~$w$ и~$b$ мы не взяли, хотя бы одно из ограничений в задаче будет нарушено, другими словами задача не имеет решений. Для этого нам необходим смягчить ограничения следующим образом:

Сделаем эти ограничения <<мягкими>>, введя штраф $\xi_i \geq 0$ за их нарушение:

$\large y_i \left(\langle w, x_i \rangle + b \right) \geq 1 - \xi_i, \quad i = 1, \dots, \ell$

При минимизации мы подбираем $w, \xi$ и возникает проблема в том, что $\xi_i$ можно устремить к бесконечости и тогда отступ может быть любым, а если отступ может быть любым, значить $w$ можно взять, например, равное нулю и это решение подойдёт, что для нас плохо:

$\large \begin{equation}
\label{eq:svmSep}
    \left\{
        \begin{aligned}
            & \|w\|_2^2 \to \min_{w, b, \xi_i} \\
            & y_i \left(
                \langle w, x_i \rangle + b
            \right) \geq 1, \quad i = 1, \dots, \ell. \\
            & \xi_i \geq 0
        \end{aligned}
    \right.
\end{equation}$

Для этого добавим к функционалу следующее слогаемое:

$\large \begin{equation}
\label{eq:svmSep}
    \left\{
        \begin{aligned}
            & \|w\|_2^2 + C \sum\limits_ {i = 1}^{\ell} \xi_i \to \min_{w, b, \xi_i} \\
            & y_i \left(
                \langle w, x_i \rangle + b
            \right) \geq 1, \quad i = 1, \dots, \ell. \\
            & \xi_i \geq 0
        \end{aligned}
    \right.
\end{equation}$

Тем самым одновременно требуем чтобы норма вектора весов была как можно меньше, то есть отступ был как можно больше. При этом, мы не допускаем того, что $\xi_i$ не было слишком большим, что не даст возможно устремить $\xi_i$ в бесконечность.

Чем больше здесь параметр $C$, тем сильнее мы будем настраиваться на обучающую выборку. Данная задача также является выпуклой и имеет единственное решение. Слогаемое $\|w\|_2^2$ имеет один и тот же масштаб если изменяется число объетов. Слогаемое $C \sum_\limits {i = 1}^{\ell} \xi_i$ изменяет масштаб если изменяется размер выборки, тем самым если мы подобрали $С$ для выборки размером $1000$, то $C$ для выборки с размером в миллион уже не подойдёт, для решения этой проблемы неходимо разделить на объём выборки:

$\large C \frac{1}{\ell} \sum\limits_ {i = 1}^{\ell} \xi_i$

<h2 style="color:#008B8B">2.3 Сведение к безусловной задаче</h2>

В начале лекции мы планировали разобраться с двумя функциями потерь: логистической и кусочно-линейной. Функционал логистической регрессии действительно имеет вид верхней оценки на долю неправильных ответов, а вот задача метода опорных векторов пока выглядит совершенно иначе. Попробуем свести её к задаче безусловной оптимизации.

Перепишем условия задачи:

$\large \begin{equation}
\label{eq:svmSep}
    \left\{
    \begin{aligned}
        &\xi_i \geq 1 - y_i (\langle w, x_i \rangle + b) \\
        &\xi_i \geq 0
    \end{aligned}
    \right.
\end{equation}$

Поскольку при этом в функционале требуется, чтобы штрафы $\xi_i$ были как можно меньше, то можно получить следующую явную формулу для них:

$\large \xi_i = \max(0, 1 - y_i (\langle w, x_i \rangle + b))$

Данной выражение для $\xi_i$ уже учитывает в себе все ограничения задачи. Значит, если подставить его в функционал, то получим безусловную задачу оптимизации:

$\large \|w\|_2^2 + C \sum\limits_{i = 1}^{\ell} \max(0, 1 - y_i (\langle w, x_i \rangle + b)) \to \underset{w, b}{\text{min}}$ 

Данная безусловная задача эквивалентна условной задаче. Заметим следующее, что $y_i (\langle w, x_i \rangle + b)$ - отступ на $i$ объекте ($M_i$). Разделим выражение на $C$:

$\large \frac{1}{\ell} \sum\limits_{i = 1}^{\ell} \max(0, 1 - M_i) + \frac{1}{C} \|w\|_2^2 \to \underset{w, b}{\text{min}}$ 

И мы получили классическую задачу линейного классификатора, где $\frac{1}{\ell} \sum\limits_{i = 1}^{\ell} \max(0, 1 - M_i)$ - функционал ошибки, а $\frac{1}{C} \|w\|_2^2$ - регуляризатор

Это формулу мы уже записывали:

$\large [M < 0] \leq max(0, 1 - M)$

**Дополнительно:**

Логистическая регрессия максимизирует отступы, то есть логистическая регрессия требует чтобы отступы на всех объектах стремились к бесконечности. Это хорошо видно на графике, логистическая регрессия никогда не достигает нуля, поэтому если мы сдвигаем отступ немного вправо тогда и ошибка станет немного меньше, поэтому миинимум достигается только в бесконечности.

В SVM мы требуем чтобы отступ был больше или равен еденицы, на графике видно что ошибка в 1 уже равна нулю и дальше не важно какой размер отступа, это не окажет влияение на ошибку.

В этом и основное отличие SVM от логистической регрессии, SVM не пытается максимизировать отступы. SVM оказывается лучше если мы оптимизируем метрику качества в кторой важно чтобы мы смогли разделить объекты, то есть оптимизация AUC-ROC, F-мера. А логистическая регрессия хорошо работает если мы хотим получать хорошие вероятности на выходе, это log-loss, оценивание вероятностей.

**Колибровка вероятностей:**

Можно визуализировать то, насколько хорошо модель оценивает вероятности. Найдём все объекты на которых модель выдала вероятность от $0$ до $0.1$ и среди них посчитаем какая доля положительных объектов (ось y) и так мы рассчитываем доли для каждой доли от $0$ до $0.1$, от $0.1$ до $0.2$...

Соеденив все точки мы получаем колибровочную кривую и она отражает то насколько хорошо модель выдаёт вероятности, если кривая находится близко к диагонали, тогда если мы берём объекты с вероятностью около $0.5$ и там 50% объктов отнесены к положительным. Значит, если кривая близка к диагонали, тогда вероятности оцениваются корректно.

Для SVM поступить можно так. Можно взять вектор весов обученный SVM от него взять сигмойду от скалярного произведения и получим числа от 0 до 1, далее можно исследовать это насколько хорошие вероятности получились. 

Зачастую колибровочная кривая для логистической регрессии распаложена возле диагонали. Для SVM может принять форму буквы S, это означает то, что у объектов с вероятностями от $0.4$ до $0.5$ большая часть отрицательные. Колибровку можно производить следующим образом, допутсим, мы знаем что у объектов с вероятность от $0.4$ до $0.5$ практически не имеится отрицательных объектов, мы можем корректировать сами вероятности $b(x)$ так, чтобы вероятность совпадала с долей полжительных объектов.